In [1]:
import { display } from "tslab";
import { readFileSync } from "fs";

const css = readFileSync("../style.css", "utf8");
display.html(`<style>${css}</style>`);

In [2]:
const { execSync } = await import('child_process');
console.log(execSync('npm install @viz-js/viz').toString());


up to date, audited 13 packages in 864ms

1 package is looking for funding
  run `npm fund` for details

found 0 vulnerabilities



Import der TypeScript Datei aus dem letzen Notebook.

In [3]:
import { NFA, DFA, nfa2dfa, TransRel } from "./01-NFA-2-DFA.js";

function showSet<T>(s: Set<T>): string {
  return `{ ${Array.from(s).join(", ")} }`;
}

const States = new Set(
  Array.from({ length: 8 }, (_, i) => `q${i}`)
);

const Sigma = new Set(["a", "b"]);

const delta: TransRel = new Map([
  ["q0,ε", new Set(["q1", "q2"])],
  ["q1,b", new Set(["q3"])],
  ["q2,a", new Set(["q4"])],
  ["q3,a", new Set(["q5"])],
  ["q4,b", new Set(["q6"])],
  ["q5,ε", new Set(["q7"])],
  ["q6,ε", new Set(["q7"])],
  ["q7,ε", new Set(["q0"])],
]);
console.log("Zustände:", showSet(States));
console.log("Alphabet:", showSet(Sigma));
console.log("Anzahl Übergänge:", delta.size);

const nfa44: NFA = {
  Q: States,
  Sigma,
  delta,
  q0: "q0",
  A: new Set(["q7"]),
};

console.log("NFA erstellt ✅");

Zustände: { q0, q1, q2, q3, q4, q5, q6, q7 }
Alphabet: { a, b }
Anzahl Übergänge: 8
NFA erstellt ✅


States = {'4', '1', '2', '3'}
δ = {
    ('4', 'a'): {'4'},
    ('4', 'b'): {'4','1'},
    ('1', 'a'): {'2'},
    ('1', 'b'): {'2'},
    ('2', 'a'): {'3'},
    ('2', 'b'): {'3'}
}

The non-deterministic <span style="font-variant:small-caps;">Fsm</span> defined below is taken from the lecture notes.

The function `nfa2dot`can be used to render this <span style="font-variant:small-caps;">Fsm</span>.

In [4]:
console.log("Starte NFA → DFA Konvertierung...");
const dfa44: DFA = nfa2dfa(nfa44);
console.log("✅ DFA erfolgreich erstellt");
console.log("Anzahl DFA-Zustände:", dfa44.Q.size);
console.log("Anzahl Akzeptierender Zustände:", dfa44.A.size);

Starte NFA → DFA Konvertierung...
✅ DFA erfolgreich erstellt
Anzahl DFA-Zustände: 6
Anzahl Akzeptierender Zustände: 2


In [5]:
console.log("DFA-Zustände:");

let i = 0;
for (const state of dfa44.Q) {
  console.log(`S${i++} = ${showSet(state)}`);
}

DFA-Zustände:
S0 = { q0, q1, q2 }
S1 = { q4 }
S2 = { q3 }
S3 = {  }
S4 = { q6, q7, q0, q1, q2 }
S5 = { q5, q7, q0, q1, q2 }


This recognizes the same language as the language described by
$$ (a \cdot b + b \cdot a) \cdot (a \cdot b + b \cdot a)^* $$
Let us convert it into a deterministic <span style="font-variant:small-caps;">Fsm</span>: 

In [27]:
import * as fs from "fs";
import { instance } from "@viz-js/viz";
import { nfa2dfa } from "./01-NFA-2-DFA.js";

function dfaToDot(dfa: any): string {
  const lines: string[] = [];
  lines.push("digraph DFA {");
  lines.push("  rankdir=LR;");
  lines.push("  node [shape = doublecircle];");
  for (const F of dfa.A) lines.push(`  "${Array.from(F).join(",")}"`);
  lines.push("  node [shape = circle];");
  for (const [key, target] of dfa.delta.entries()) {
    const [fromKey, c] = key.split(/,(?=[^,]*$)/);
    const from = fromKey.replace(/[\[\]"]/g, "");
    const to = Array.from(target).join(",");
    lines.push(`  "${from}" -> "${to}" [label="${c}"];`);
  }
  lines.push("}");
  return lines.join("\n");
}

async function saveDFAasSVG(dfa: any, filename = "dfa.svg") {
  // 1️⃣ WebAssembly laden
  const viz = await instance();

  // 2️⃣ DOT erzeugen
  const dot = dfaToDot(dfa);

  // 3️⃣ SVG rendern (hier passiert die Umwandlung!)
  const svg = viz.renderString(dot, { format: "svg" });

  display.html(svg);
}

// Beispiel
const dfa = nfa2dfa(nfa44);
await saveDFAasSVG(dfa);


<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 14.0.1 (20251006.0113)
 -->
<!-- Title: DFA Pages: 1 -->
 
 
 DFA 
 
<!-- q6,q7,q0,q1,q2 -->
 
 q6,q7,q0,q1,q2 
 
 
 q6,q7,q0,q1,q2 
 
<!-- q5,q7,q0,q1,q2 -->
 
 q5,q7,q0,q1,q2 
 
 
 q5,q7,q0,q1,q2 
 
<!-- q0,q1,q2 -->
 
 q0,q1,q2 
 
 q0,q1,q2 
 
<!-- q4 -->
 
 q4 
 
 q4 
 
<!-- q0,q1,q2->q4 -->
 
 q0,q1,q2->q4 
 
 
 a 
 
<!-- q3 -->
 
 q3 
 
 q3 
 
<!-- q0,q1,q2->q3 -->
 
 q0,q1,q2->q3 
 
 
 b 
 
<!-- q4->q6,q7,q0,q1,q2 -->
 
 q4->q6,q7,q0,q1,q2 
 
 
 b 
 
 
 
 
 
<!-- q4-> -->
 
 q4-> 
 
 
 a 
 
<!-- q3->q5,q7,q0,q1,q2 -->
 
 q3->q5,q7,q0,q1,q2 
 
 
 a 
 
<!-- q3-> -->
 
 q3-> 
 
 
 b 
 
<!-- -> -->
 
 -> 
 
 
 a 
 
<!-- -> -->
 
 -> 
 
 
 b 
 
<!-- q0,q1,q2,q6,q7 -->
 
 q0,q1,q2,q6,q7 
 
 q0,q1,q2,q6,q7 
 
<!-- q0,q1,q2,q6,q7->q4 -->
 
 q0,q1,q2,q6,q7->q4 
 
 
 a 
 
<!-- q0,q1,q2,q6,q7->q3 -->
 
 q0,q1,q2,q6,q7->q3 
 
 
 b 
 
<!-- q0,q1,q2,q5,q7 -->
 
 q0,q1,q2,q5,q7 
 
 q0,q1,q2,q5,q7 
 
<!-- q0,q1,q2,q5,q7->q4 -->
 
 q0,q1,q2,q5,q7->q4 
 
 
 a 
 
<!-- q0,q1,q2,q5,q7->q3 -->
 
 q0,q1,q2,q5,q7->q3 
 
 
 b

UncaughtException: Error: Unexpected pending rebuildTimer
    at sys.setTimeout (C:\Users\ertan\anaconda3\envs\fl\node_modules\tslab\dist\converter.js:111:19)
    at scheduleProgramUpdate (C:\Users\ertan\anaconda3\envs\fl\node_modules\tslab\node_modules\@tslab\typescript-for-tslab\lib\typescript.js:122735:35)
    at onSourceFileChange (C:\Users\ertan\anaconda3\envs\fl\node_modules\tslab\node_modules\@tslab\typescript-for-tslab\lib\typescript.js:122876:7)
    at C:\Users\ertan\anaconda3\envs\fl\node_modules\tslab\node_modules\@tslab\typescript-for-tslab\lib\typescript.js:122868:56
    at cb (C:\Users\ertan\anaconda3\envs\fl\node_modules\tslab\dist\converter.js:184:13)
    at C:\Users\ertan\anaconda3\envs\fl\node_modules\tslab\node_modules\@tslab\typescript-for-tslab\lib\typescript.js:5798:9
    at C:\Users\ertan\anaconda3\envs\fl\node_modules\tslab\node_modules\@tslab\typescript-for-tslab\lib\typescript.js:5560:101
    at Array.forEach (<anonymous>)
    at FSWatcher.<anonymous> (C:\User

UncaughtException: Error: Unexpected pending rebuildTimer
    at sys.setTimeout (C:\Users\ertan\anaconda3\envs\fl\node_modules\tslab\dist\converter.js:111:19)
    at scheduleProgramUpdate (C:\Users\ertan\anaconda3\envs\fl\node_modules\tslab\node_modules\@tslab\typescript-for-tslab\lib\typescript.js:122735:35)
    at onSourceFileChange (C:\Users\ertan\anaconda3\envs\fl\node_modules\tslab\node_modules\@tslab\typescript-for-tslab\lib\typescript.js:122876:7)
    at C:\Users\ertan\anaconda3\envs\fl\node_modules\tslab\node_modules\@tslab\typescript-for-tslab\lib\typescript.js:122868:56
    at cb (C:\Users\ertan\anaconda3\envs\fl\node_modules\tslab\dist\converter.js:184:13)
    at C:\Users\ertan\anaconda3\envs\fl\node_modules\tslab\node_modules\@tslab\typescript-for-tslab\lib\typescript.js:5798:9
    at C:\Users\ertan\anaconda3\envs\fl\node_modules\tslab\node_modules\@tslab\typescript-for-tslab\lib\typescript.js:5560:101
    at Array.forEach (<anonymous>)
    at FSWatcher.<anonymous> (C:\User

UncaughtException: Error: Unexpected pending rebuildTimer
    at sys.setTimeout (C:\Users\ertan\anaconda3\envs\fl\node_modules\tslab\dist\converter.js:111:19)
    at scheduleProgramUpdate (C:\Users\ertan\anaconda3\envs\fl\node_modules\tslab\node_modules\@tslab\typescript-for-tslab\lib\typescript.js:122735:35)
    at onSourceFileChange (C:\Users\ertan\anaconda3\envs\fl\node_modules\tslab\node_modules\@tslab\typescript-for-tslab\lib\typescript.js:122876:7)
    at C:\Users\ertan\anaconda3\envs\fl\node_modules\tslab\node_modules\@tslab\typescript-for-tslab\lib\typescript.js:122868:56
    at cb (C:\Users\ertan\anaconda3\envs\fl\node_modules\tslab\dist\converter.js:184:13)
    at C:\Users\ertan\anaconda3\envs\fl\node_modules\tslab\node_modules\@tslab\typescript-for-tslab\lib\typescript.js:5798:9
    at C:\Users\ertan\anaconda3\envs\fl\node_modules\tslab\node_modules\@tslab\typescript-for-tslab\lib\typescript.js:5560:101
    at Array.forEach (<anonymous>)
    at FSWatcher.<anonymous> (C:\User

UncaughtException: Error: Unexpected pending rebuildTimer
    at sys.setTimeout (C:\Users\ertan\anaconda3\envs\fl\node_modules\tslab\dist\converter.js:111:19)
    at scheduleProgramUpdate (C:\Users\ertan\anaconda3\envs\fl\node_modules\tslab\node_modules\@tslab\typescript-for-tslab\lib\typescript.js:122735:35)
    at onSourceFileChange (C:\Users\ertan\anaconda3\envs\fl\node_modules\tslab\node_modules\@tslab\typescript-for-tslab\lib\typescript.js:122876:7)
    at C:\Users\ertan\anaconda3\envs\fl\node_modules\tslab\node_modules\@tslab\typescript-for-tslab\lib\typescript.js:122868:56
    at cb (C:\Users\ertan\anaconda3\envs\fl\node_modules\tslab\dist\converter.js:184:13)
    at C:\Users\ertan\anaconda3\envs\fl\node_modules\tslab\node_modules\@tslab\typescript-for-tslab\lib\typescript.js:5798:9
    at C:\Users\ertan\anaconda3\envs\fl\node_modules\tslab\node_modules\@tslab\typescript-for-tslab\lib\typescript.js:5560:101
    at Array.forEach (<anonymous>)
    at FSWatcher.<anonymous> (C:\User

...

UncaughtException: Error: Unexpected pending rebuildTimer
    at sys.setTimeout (C:\Users\ertan\anaconda3\envs\fl\node_modules\tslab\dist\converter.js:111:19)
    at scheduleProgramUpdate (C:\Users\ertan\anaconda3\envs\fl\node_modules\tslab\node_modules\@tslab\typescript-for-tslab\lib\typescript.js:122735:35)
    at onSourceFileChange (C:\Users\ertan\anaconda3\envs\fl\node_modules\tslab\node_modules\@tslab\typescript-for-tslab\lib\typescript.js:122876:7)
    at C:\Users\ertan\anaconda3\envs\fl\node_modules\tslab\node_modules\@tslab\typescript-for-tslab\lib\typescript.js:122868:56
    at cb (C:\Users\ertan\anaconda3\envs\fl\node_modules\tslab\dist\converter.js:184:13)
    at C:\Users\ertan\anaconda3\envs\fl\node_modules\tslab\node_modules\@tslab\typescript-for-tslab\lib\typescript.js:5798:9
    at C:\Users\ertan\anaconda3\envs\fl\node_modules\tslab\node_modules\@tslab\typescript-for-tslab\lib\typescript.js:5560:101
    at Array.forEach (<anonymous>)
    at FSWatcher.<anonymous> (C:\User

UncaughtException: Error: Unexpected pending rebuildTimer
    at sys.setTimeout (C:\Users\ertan\anaconda3\envs\fl\node_modules\tslab\dist\converter.js:111:19)
    at scheduleProgramUpdate (C:\Users\ertan\anaconda3\envs\fl\node_modules\tslab\node_modules\@tslab\typescript-for-tslab\lib\typescript.js:122735:35)
    at onSourceFileChange (C:\Users\ertan\anaconda3\envs\fl\node_modules\tslab\node_modules\@tslab\typescript-for-tslab\lib\typescript.js:122876:7)
    at C:\Users\ertan\anaconda3\envs\fl\node_modules\tslab\node_modules\@tslab\typescript-for-tslab\lib\typescript.js:122868:56
    at cb (C:\Users\ertan\anaconda3\envs\fl\node_modules\tslab\dist\converter.js:184:13)
    at C:\Users\ertan\anaconda3\envs\fl\node_modules\tslab\node_modules\@tslab\typescript-for-tslab\lib\typescript.js:5798:9
    at C:\Users\ertan\anaconda3\envs\fl\node_modules\tslab\node_modules\@tslab\typescript-for-tslab\lib\typescript.js:5560:101
    at Array.forEach (<anonymous>)
    at FSWatcher.<anonymous> (C:\User

UncaughtException: Error: Unexpected pending rebuildTimer
    at sys.setTimeout (C:\Users\ertan\anaconda3\envs\fl\node_modules\tslab\dist\converter.js:111:19)
    at scheduleProgramUpdate (C:\Users\ertan\anaconda3\envs\fl\node_modules\tslab\node_modules\@tslab\typescript-for-tslab\lib\typescript.js:122735:35)
    at onSourceFileChange (C:\Users\ertan\anaconda3\envs\fl\node_modules\tslab\node_modules\@tslab\typescript-for-tslab\lib\typescript.js:122876:7)
    at C:\Users\ertan\anaconda3\envs\fl\node_modules\tslab\node_modules\@tslab\typescript-for-tslab\lib\typescript.js:122868:56
    at cb (C:\Users\ertan\anaconda3\envs\fl\node_modules\tslab\dist\converter.js:184:13)
    at C:\Users\ertan\anaconda3\envs\fl\node_modules\tslab\node_modules\@tslab\typescript-for-tslab\lib\typescript.js:5798:9
    at C:\Users\ertan\anaconda3\envs\fl\node_modules\tslab\node_modules\@tslab\typescript-for-tslab\lib\typescript.js:5560:101
    at Array.forEach (<anonymous>)
    at FSWatcher.<anonymous> (C:\User

UncaughtException: Error: Unexpected pending rebuildTimer
    at sys.setTimeout (C:\Users\ertan\anaconda3\envs\fl\node_modules\tslab\dist\converter.js:111:19)
    at scheduleProgramUpdate (C:\Users\ertan\anaconda3\envs\fl\node_modules\tslab\node_modules\@tslab\typescript-for-tslab\lib\typescript.js:122735:35)
    at onSourceFileChange (C:\Users\ertan\anaconda3\envs\fl\node_modules\tslab\node_modules\@tslab\typescript-for-tslab\lib\typescript.js:122876:7)
    at C:\Users\ertan\anaconda3\envs\fl\node_modules\tslab\node_modules\@tslab\typescript-for-tslab\lib\typescript.js:122868:56
    at cb (C:\Users\ertan\anaconda3\envs\fl\node_modules\tslab\dist\converter.js:184:13)
    at C:\Users\ertan\anaconda3\envs\fl\node_modules\tslab\node_modules\@tslab\typescript-for-tslab\lib\typescript.js:5798:9
    at C:\Users\ertan\anaconda3\envs\fl\node_modules\tslab\node_modules\@tslab\typescript-for-tslab\lib\typescript.js:5560:101
    at Array.forEach (<anonymous>)
    at FSWatcher.<anonymous> (C:\User

UncaughtException: Error: Unexpected pending rebuildTimer
    at sys.setTimeout (C:\Users\ertan\anaconda3\envs\fl\node_modules\tslab\dist\converter.js:111:19)
    at scheduleProgramUpdate (C:\Users\ertan\anaconda3\envs\fl\node_modules\tslab\node_modules\@tslab\typescript-for-tslab\lib\typescript.js:122735:35)
    at onSourceFileChange (C:\Users\ertan\anaconda3\envs\fl\node_modules\tslab\node_modules\@tslab\typescript-for-tslab\lib\typescript.js:122876:7)
    at C:\Users\ertan\anaconda3\envs\fl\node_modules\tslab\node_modules\@tslab\typescript-for-tslab\lib\typescript.js:122868:56
    at cb (C:\Users\ertan\anaconda3\envs\fl\node_modules\tslab\dist\converter.js:184:13)
    at C:\Users\ertan\anaconda3\envs\fl\node_modules\tslab\node_modules\@tslab\typescript-for-tslab\lib\typescript.js:5798:9
    at C:\Users\ertan\anaconda3\envs\fl\node_modules\tslab\node_modules\@tslab\typescript-for-tslab\lib\typescript.js:5560:101
    at Array.forEach (<anonymous>)
    at FSWatcher.<anonymous> (C:\User

The function `dfa2dot`can be used to render this <span style="font-variant:small-caps;">Fsm</span>.

In order to inspect the states of this deterministic <span style="font-variant:small-caps;">Fsm</span> we print the dictionary  `states2Names`.

In [ ]:
states2Names

We can also print the <span style="font-variant:small-caps;">Fsm</span>.

In [ ]:
dfa44